In [8]:
pip install openpyxl
pip install yfinance
pip install pandas
pip install numpy

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\raip0\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import yfinance as yf
import numpy as np
import json
import openpyxl

In [2]:
# stock info

def get_currentprice(symbol):
    ticker = yf.Ticker(symbol)
    current_price = ticker.info['currentPrice']
    return current_price

def get_marketcap(symbol):
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info['marketCap']
    return market_cap

def get_industry(symbol):
    ticker = yf.Ticker(symbol)
    industry = ticker.info['industry']
    return industry

In [3]:
# multiples functions

def get_ev(symbol):
    ticker = yf.Ticker(symbol)

    market_cap = ticker.info['marketCap']
    balance_sheet = ticker.balance_sheet
    last_total_debt = balance_sheet.loc['Total Debt'].iloc[0]
    last_cash_and_eq = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
    ev = market_cap + last_total_debt - last_cash_and_eq

    return ev

def get_pe_ratio(symbol):
    ticker = yf.Ticker(symbol)
    current_price = ticker.info['currentPrice']

    income_statement = ticker.quarterly_financials
    last_net_income = income_statement.loc['Net Income'].iloc[:4].sum()

    balance_sheet = ticker.balance_sheet
    last_shares_number = balance_sheet.loc['Ordinary Shares Number'].iloc[0]

    eps_ratio = last_net_income / last_shares_number
    pe_ratio = current_price / eps_ratio
    return pe_ratio

def get_evebitda_ratio(symbol):
    ticker = yf.Ticker(symbol)

    market_cap = ticker.info['marketCap']
    balance_sheet = ticker.balance_sheet
    last_total_debt = balance_sheet.loc['Total Debt'].iloc[0]
    last_cash_and_eq = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
    ev = market_cap + last_total_debt - last_cash_and_eq

    income_statement = ticker.quarterly_financials
    if 'EBITDA' in income_statement.index:
        ebitda = income_statement.loc['EBITDA'].iloc[:4].sum()
    else:
        return None
    
    evebitda_ratio = ev / ebitda
    return evebitda_ratio

def get_evfcf_ratio(symbol):
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info['marketCap']
    balance_sheet = ticker.balance_sheet
    last_total_debt = balance_sheet.loc['Total Debt'].iloc[0]
    last_cash_and_eq = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]

    ev = market_cap + last_total_debt - last_cash_and_eq

    cash_flow = ticker.quarterly_cash_flow
    last_free_cash_flow = cash_flow.loc['Free Cash Flow'].iloc[:4].sum()

    evfcf_ratio = ev / last_free_cash_flow

    return evfcf_ratio


def get_pb_ratio(symbol):
    ticker = yf.Ticker(symbol)
    current_price = ticker.info['currentPrice']

    balance_sheet = ticker.balance_sheet
    last_equity = balance_sheet.loc['Total Equity Gross Minority Interest'].iloc[0]
    last_shares_number = balance_sheet.loc['Ordinary Shares Number'].iloc[0]
    bookvalue_pershare = last_equity / last_shares_number

    pb_ratio = current_price / bookvalue_pershare

    return pb_ratio

def get_evrevenue_ratio(symbol):
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info['marketCap']
    balance_sheet = ticker.balance_sheet
    last_total_debt = balance_sheet.loc['Total Debt'].iloc[0]
    last_cash_and_eq = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]

    ev = market_cap + last_total_debt - last_cash_and_eq

    income_statement = ticker.quarterly_financials
    last_revenue = income_statement.loc['Total Revenue'].iloc[:4].sum()

    evrevenue_ratio = ev / last_revenue

    return evrevenue_ratio


In [4]:
# avg growth rates
def percentage_changes(data):
    if len(data) < 2:
        return []  # not enough elements 

    percentage_differences = []
    for i in range(len(data) - 1):
        value1 = data[i]
        value2 = data[i + 1]
        # calculate percentage difference
        if value2 < 0:
            difference = -((value1 / value2) - 1) # en este tipo de formula, cuando el valor antiguo es negativo el resultado es incorrecto ya que da un crecimiento negativo, por eso se agrega el signo de menos al principio
        else:
            difference = (value1 / value2) - 1 
        percentage_differences.append(difference)
    
    return percentage_differences

def get_eps_growthrate(symbol):
    ticker = yf.Ticker(symbol)
    net_income = list(ticker.financials.loc['Basic EPS'].iloc[:4])

    percent_change = np.mean(percentage_changes(net_income))
    return percent_change

def get_fcf_growthrate(symbol):
    ticker = yf.Ticker(symbol)
    fcf = list(ticker.cash_flow.loc['Free Cash Flow'].iloc[:4])
    percent_change = np.mean(percentage_changes(fcf))

    return percent_change




In [5]:
def mult_valuation(tickers_lst):
    data = []

    for ticker in tickers_lst:
        current_price = get_currentprice(ticker)
        market_cap = get_marketcap(ticker)
        pe_ratio = get_pe_ratio(ticker)
        pb_ratio = get_pb_ratio(ticker)
        ev_revenue = get_evrevenue_ratio(ticker)
        ev_ebitda = get_evebitda_ratio(ticker)
        ev_fcf = get_evfcf_ratio(ticker)
        fcf_gr = get_fcf_growthrate(ticker)
        eps_gr = get_eps_growthrate(ticker)
        industry = get_industry(ticker)
        
        data.append({
            'Ticker': ticker,
            'Current Price': current_price,
            'Market Cap': market_cap,
            'P/E Ratio': pe_ratio,
            'P/B Ratio': pb_ratio,
            'EV/Revenue': ev_revenue,
            'EV/EBITDA': ev_ebitda,
            'EV/FCF Ratio': ev_fcf,
            'FCF Growth Rate': fcf_gr,
            'EPS Growth Rate': eps_gr,
            'Industry': industry
        })

    df = pd.DataFrame(data)
    return df

In [10]:
def df_to_excel(ticker_groups):
    # nuevo arch excel
    with pd.ExcelWriter('valuation_summary.xlsx') as writer:
        for idx, tickers in enumerate(ticker_groups):
            # df por grupo
            group_df = mult_valuation(tickers)
            
            # dataframe por sheet
            sheet_name = f'Group {idx + 1}'
            group_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print("All dataframes saved to valuation_summary.xlsx")

ticker_groups = [
    ['AAPL', 'AMZN', 'MSFT', 'GOOG', 'META', 'NVDA'],  
    ['WFC', 'JPM', 'C', 'MS', 'GS', 'AXP'],
    ['MCD', 'SBUX', 'CMG', 'DPZ', 'TXRH'],
    ['CAT', 'DE', 'PCAR', 'CNH', 'OSK', 'REVG'],
    ['TMUS', 'VZ', 'T', 'CMCSA'],
    ['NVDA', 'AVGO', 'TSM', 'AMD', 'QCOM'],
    ['TPL', 'COP', 'EOG', 'FANG', 'OXY'],
    ['ASML', 'AMAT', 'LRCX', 'KLAC', 'TER'],
    ['BX', 'BLK', 'KKR', 'APO', 'ARES'],
    ['SPGI', 'MCO', 'ICE', 'CME', 'MSCI', 'NDAQ']
]

df_to_excel(ticker_groups)

All dataframes saved to valuation_summary.xlsx
